In [1]:
#импортируем библиотеку pandas
import pandas as pd

In [2]:
#загружаем датасет
pokemon_df = pd.read_csv('../data/pokemon.csv')

In [3]:
#убеждаемся, что все правильно загрузили и заодно смотрим на часть данных в датасете
pokemon_df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [4]:
#смотрим информацию о датасете
pokemon_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


Из данной информации видно, что пропущенные значения есть только в столбце признака "Type 2". В данном случае значения пропущены не из-за того, что о них нет информации, а из-за того, что они попросту отсутствуют у некоторых объектов.

In [5]:
pokemon_df['Type 2'].unique()

array(['Poison', nan, 'Flying', 'Dragon', 'Ground', 'Fairy', 'Grass',
       'Fighting', 'Psychic', 'Steel', 'Ice', 'Rock', 'Dark', 'Water',
       'Electric', 'Fire', 'Ghost', 'Bug', 'Normal'], dtype=object)

In [6]:
pokemon_df['Type 1'].unique()

array(['Grass', 'Fire', 'Water', 'Bug', 'Normal', 'Poison', 'Electric',
       'Ground', 'Fairy', 'Fighting', 'Psychic', 'Rock', 'Ghost', 'Ice',
       'Dragon', 'Dark', 'Steel', 'Flying'], dtype=object)

Информация, приведенная сверху, говорит нам о том, что оба признака "Type 1" и "Type 2" принимают значения из одного множества значений. Планируется предсказывать именно первый тип по числовым характеристикам покемона, поэтому столбец "Type 2" можно смело удалить.

In [7]:
#удаляем столбец Type 2
pokemon_df = pokemon_df.drop(['Type 2'], axis=1)
pokemon_df.head()

,#,Name,Type 1,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,309,39,52,43,60,50,65,1,False


Теперь, когда в датафрейме нет пустых значений, необходимо перекодировать категориальные признаки. В данном датасете имеется два категориальных признака: "Name" и "Type 1". Признак "Name" не является информативным, поэтому его тоже можно удалить. Так как Type 1 является целевым признаком, его мы кодировать не будем.

In [8]:
#удаляем столбец Name
pokemon_df = pokemon_df.drop(['Name'], axis=1)

In [9]:
#проверяем датафрейм
pokemon_df.head()

,#,Type 1,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Grass,318,45,49,49,65,65,45,1,False
1,2,Grass,405,60,62,63,80,80,60,1,False
2,3,Grass,525,80,82,83,100,100,80,1,False
3,3,Grass,625,80,100,123,122,120,80,1,False
4,4,Fire,309,39,52,43,60,50,65,1,False


Теперь необходимо закодировать порядковые признаки. В датафрейме имеется два порядковых признака: "#" и "Generation". "#" не является информативным признаком, поэтому его можно удалить. 

In [10]:
#удаляем столбец '#'
pokemon_df = pokemon_df.drop(['#'], axis=1)

Признак "Generation" закодируем one hot методом.

In [11]:
#one-hot кодирование
pokemon_df = pd.get_dummies(pokemon_df, prefix=['gen'], columns=['Generation'])

In [12]:
pokemon_df.head()

,Type 1,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary,gen_1,gen_2,gen_3,gen_4,gen_5,gen_6
0,Grass,318,45,49,49,65,65,45,False,1,0,0,0,0,0
1,Grass,405,60,62,63,80,80,60,False,1,0,0,0,0,0
2,Grass,525,80,82,83,100,100,80,False,1,0,0,0,0,0
3,Grass,625,80,100,123,122,120,80,False,1,0,0,0,0,0
4,Fire,309,39,52,43,60,50,65,False,1,0,0,0,0,0


Обработаем бинарные признаки. Он у нас всего один - 'Legendary'. Переведем False/True в 0/1.

In [13]:
pokemon_df['Legendary'] = pokemon_df['Legendary'].astype('int64', copy=False)

In [14]:
pokemon_df

,Type 1,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary,gen_1,gen_2,gen_3,gen_4,gen_5,gen_6
0,Grass,318,45,49,49,65,65,45,0,1,0,0,0,0,0
1,Grass,405,60,62,63,80,80,60,0,1,0,0,0,0,0
2,Grass,525,80,82,83,100,100,80,0,1,0,0,0,0,0
3,Grass,625,80,100,123,122,120,80,0,1,0,0,0,0,0
4,Fire,309,39,52,43,60,50,65,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,Rock,600,50,100,150,100,150,50,1,0,0,0,0,0,1
796,Rock,700,50,160,110,160,110,110,1,0,0,0,0,0,1
797,Psychic,600,80,110,60,150,130,70,1,0,0,0,0,0,1
798,Psychic,680,80,160,60,170,130,80,1,0,0,0,0,0,1


In [15]:
#экспортируем изменения в датафрейме
pokemon_df.to_csv('../data/pokemon_preprocessed.csv', index=False)